In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
import geopandas as gp
import yaml
import math

from scipy.spatial import ConvexHull
from scipy.interpolate import griddata
from shapely.geometry import Point, Polygon

from pykrige.ok import OrdinaryKriging
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from pyproj import Proj
from scipy.io import savemat

/tmp/ipykernel_731416/2798409764.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'pykrige'

In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

: 

In [ ]:
flag_project = 1 # 0--regular; 1--repartrition

: 

In [ ]:
if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']

#name_project = 'project/output_FJSJ_16-01/'               
#name_project = 'project_repartrition/repartrition_01-03/'               
#name_project = 'project_voronoi/voronoi_01-03/'         

: 

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

: 

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

: 

In [ ]:
dir_model_fund = dir_project + info_basic['dir_model_fund']
dir_model = dir_project + info_basic['dir_model']
dir_image = dir_project + info_basic['dir_image']+'Vs_compare/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_inv_dispernet = dir_project + info_basic['dir_inv_dispernet']
dir_partition = dir_project + info_basic['dir_partition']
key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

: 

In [ ]:
dir_file = 'ModelValidCode/ModelValidCode/parameter_prepare/'

: 

In [ ]:
def GetStationPairs(nsta):
    StationPair = []
    for ii in range(nsta):
        for jj in range(ii+1,nsta):
            StationPair.append(ii)
            StationPair.append(jj)
    StationPair = np.array(StationPair,dtype=np.int32)
    return StationPair

: 

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 
elevation_stations_all = stainfo['elevation'].tolist()

: 

In [ ]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
num_stations = []
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    num_stations.append(len(stations_this))
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

: 

In [ ]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

: 

### Interpolate 3D structure

In [ ]:
dz = 3
N = 68
num_x = 50
num_y = 50
type_interp = 'linear'
flag_save = 1 # 1--npz; 2--mat

: 

In [ ]:
# Read all data
struc = {}
struc_fund = {}
flag = 0
for key_subwork in key_subworks:
    file_model = dir_model + 'model_'+key_subwork+'.txt'
    file_model_fund = dir_model_fund + 'model_'+key_subwork+'.txt'
    model = np.loadtxt(file_model)
    model_fund = np.loadtxt(file_model_fund)
    struc[key_subwork] = {}
    struc[key_subwork]['layer'] = model[:, 0]
    struc[key_subwork]['z'] = model[:, 1]
    struc[key_subwork]['rho'] = model[:, 2]
    struc[key_subwork]['vs'] = model[:, 3]
    struc[key_subwork]['vp'] = model[:, 4]
    struc[key_subwork]['std'] = model[:, 5]
    struc_fund[key_subwork] = {}
    struc_fund[key_subwork]['layer'] = model_fund[:, 0]
    struc_fund[key_subwork]['z'] = model_fund[:, 1]
    struc_fund[key_subwork]['rho'] = model_fund[:, 2]
    struc_fund[key_subwork]['vs'] = model_fund[:, 3]
    struc_fund[key_subwork]['vp'] = model_fund[:, 4]
    struc_fund[key_subwork]['std'] = model_fund[:, 5]
    flag += 1
    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))

vs_horizon = {}
vs_horizon_fund = {}
for i in range(N):
    vs_horizon[i] = []
    vs_horizon_fund[i] = []
    for key_subwork in key_subworks:
        vs_horizon[i].append(struc[key_subwork]['vs'][i])
        vs_horizon_fund[i].append(struc_fund[key_subwork]['vs'][i])

: 

In [ ]:
#points = np.column_stack((lon_stations_all, lat_stations_all))
points = np.column_stack((lon_centroid_partition, lat_centroid_partition))
hull = ConvexHull(points)
polygon = Polygon(points[hull.vertices])
index_sta = []
lon_stations_in = []
lat_stations_in = []
elevation_stations_in = []
for i in range(len(lon_stations_all)):
    if polygon.contains(Point(lon_stations_all[i], lat_stations_all[i])):
        index_sta.append(i)
        lon_stations_in.append(lon_stations_all[i])
        lat_stations_in.append(lat_stations_all[i])
        elevation_stations_in.append(elevation_stations_all[i])

: 

In [ ]:
def compute_affine_transform(original_points, target_points):
    A_matrix = np.array([[original_points[0][0], original_points[0][1], 1, 0, 0, 0],
                         [0, 0, 0, original_points[0][0], original_points[0][1], 1],
                         [original_points[1][0], original_points[1][1], 1, 0, 0, 0],
                         [0, 0, 0, original_points[1][0], original_points[1][1], 1],
                         [original_points[2][0], original_points[2][1], 1, 0, 0, 0],
                         [0, 0, 0, original_points[2][0], original_points[2][1], 1]])

    A1_B1_C1 = np.array([target_points[0][0], target_points[0][1], target_points[1][0], target_points[1][1], target_points[2][0], target_points[2][1]])

    coefficients = np.linalg.solve(A_matrix, A1_B1_C1)

    affine_matrix = np.array([[coefficients[0], coefficients[1], coefficients[2]],
                               [coefficients[3], coefficients[4], coefficients[5]],
                               [0, 0, 1]])

    return affine_matrix

: 

In [ ]:
# Affine transformation
lon_stations_all_new = []
lat_stations_all_new = []
for sta in stalist_all:
    if int(sta[1:3]) <= 60:
        lon_stations_all_new.append(lon_stations_all[stalist_all.index(sta)])
        lat_stations_all_new.append(lat_stations_all[stalist_all.index(sta)])
refs = ['R0101','R6001','R6020']
lon_refs = [lon_stations_all[stalist_all.index(ref)] for ref in refs]
lat_refs = [lat_stations_all[stalist_all.index(ref)] for ref in refs]
loc_refs = np.column_stack([lon_refs,lat_refs])
loc_refs_new = np.array([[0,0],[600,0],[600,600]])

affine_matrix = compute_affine_transform(loc_refs, loc_refs_new)

: 

In [ ]:
# all stations
x_stations_all = []
y_stations_all = []
for i in range(len(lon_stations_all)):
    loc_sta = np.array([lon_stations_all[i],lat_stations_all[i],1])
    loc_sta_new = np.dot(affine_matrix,loc_sta)
    x_stations_all.append(loc_sta_new[0])
    y_stations_all.append(loc_sta_new[1])
# new stations
x_stations_all_new = []
y_stations_all_new = []
for i in range(len(lon_stations_all_new)):
    loc_sta = np.array([lon_stations_all_new[i],lat_stations_all_new[i],1])
    loc_sta_new = np.dot(affine_matrix,loc_sta)
    x_stations_all_new.append(loc_sta_new[0])
    y_stations_all_new.append(loc_sta_new[1])
# in stations
x_stations_in = []
y_stations_in = []
for i in range(len(lon_stations_in)):
    loc_sta = np.array([lon_stations_in[i],lat_stations_in[i],1])
    loc_sta_new = np.dot(affine_matrix,loc_sta)
    x_stations_in.append(loc_sta_new[0])
    y_stations_in.append(loc_sta_new[1])
# faults
x_faults = {}
y_faults = {}
for i in range(len(faults)):
    x_faults['clark'+str(i+1)] = []
    y_faults['clark'+str(i+1)] = []
    for j in range(len(faults['clark'+str(i+1)]['lon'])):
        loc_fault = np.array([faults['clark'+str(i+1)]['lon'][j],faults['clark'+str(i+1)]['lat'][j],1])
        loc_fault_new = np.dot(affine_matrix,loc_fault)
        x_faults['clark'+str(i+1)].append(loc_fault_new[0])
        y_faults['clark'+str(i+1)].append(loc_fault_new[1])
# partitions
x_centroid_partition = []
y_centroid_partition = []
for i in range(len(lon_centroid_partition)):
    loc_centroid = np.array([lon_centroid_partition[i],lat_centroid_partition[i],1])
    loc_centroid_new = np.dot(affine_matrix,loc_centroid)
    x_centroid_partition.append(loc_centroid_new[0])
    y_centroid_partition.append(loc_centroid_new[1])

: 

In [ ]:
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(121)
ax.scatter(lon_stations_all_new, lat_stations_all_new, c='k', label='Original',s = 1)
for ref in refs:
    ax.scatter(lon_stations_all[stalist_all.index(ref)], lat_stations_all[stalist_all.index(ref)], c='g', label='Original',s = 10)
for i in range(len(faults)):
    ax.plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax = fig.add_subplot(122)
ax.scatter(x_stations_all_new, y_stations_all_new, c='k', label='Original',s = 1)
for i in range(len(faults)):
    ax.plot(x_faults['clark'+str(i+1)], y_faults['clark'+str(i+1)], 'k')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_xlim(-50,650)
ax.set_ylim(-50,650)
plt.tight_layout()

: 

### Interpolate

In [ ]:
# generate grid
xx = np.linspace(np.min(x_centroid_partition), np.max(y_centroid_partition), num_x)
yy = np.linspace(np.min(y_centroid_partition), np.max(y_centroid_partition), num_y)
A,B = np.meshgrid(xx, yy)
X_star = np.hstack((A.flatten()[:,None], B.flatten()[:,None]))
x_grid = X_star[:,0]
y_grid = X_star[:,1]

: 

In [ ]:
vs_inter_fund = np.zeros([len(xx),len(yy),N])
vs_inter = np.zeros([len(xx),len(yy),N])
for i in range(N):
    print('Interpolating horizon %d/%d' % (i+1,N))
    OK = OrdinaryKriging(x_centroid_partition, y_centroid_partition, vs_horizon[i], variogram_model=type_interp,nlags=3)
    zz,ss = OK.execute('grid', xx, yy)
    vs_inter[:,:,i] = zz
    OK = OrdinaryKriging(x_centroid_partition, y_centroid_partition, vs_horizon_fund[i], variogram_model=type_interp,nlags=3)
    zz,ss = OK.execute('grid', xx, yy)
    vs_inter_fund[:,:,i] = zz

: 

### Save

In [ ]:
def cartesian_to_geographic_2d(x, y):
    # 定义一个适当的投影，根据你的数据选择合适的投影方式
    # 这里假设使用 WGS 84 椭球体投影
    proj = Proj(proj='utm', zone=10, ellps='WGS84')
    
    # 将笛卡尔坐标系转换为经纬度
    lon, lat = proj(x, y, inverse=True)
    
    return lon, lat

: 

In [ ]:
# save as npz
z = np.arange(0,N*dz,dz)
print('Save as mat')
lons = np.zeros_like(xx)
lats = np.zeros_like(yy)
for i in range(len(xx)):
    lon, lat = cartesian_to_geographic_2d(xx[i], yy[i])
    lons[i] = lon
    lats[i] = lat
lon_stations = np.zeros_like(x_stations_in)
lat_stations = np.zeros_like(y_stations_in)
for i in range(len(x_stations_in)):
    lon, lat = cartesian_to_geographic_2d(x_stations_in[i], y_stations_in[i])
    lon_stations[i] = lon
    lat_stations[i] = lat
filename = dir + 'Vs_inter.mat'
savemat(filename, {'vs_inter': vs_inter, 'x': lons, 'y': lats, 'z': z, 'lon_sta': lon_stations, 'lat_sta': lat_stations})
filename_fund = dir_file + 'Vs_inter_fund.mat'
savemat(filename_fund, {'vs_inter_fund': vs_inter_fund, 'x': lons, 'y': lats, 'z': z, 'lon_sta': lon_stations, 'lat_sta': lat_stations})

: 

In [ ]:
# save station in
lon_stations_in_new = np.zeros_like(x_stations_in)
lat_stations_in_new = np.zeros_like(y_stations_in)
for i in range(len(x_stations_in)):
    lon, lat = cartesian_to_geographic_2d(x_stations_in[i], y_stations_in[i])
    lon_stations_in_new[i] = lon
    lat_stations_in_new[i] = lat
filename = dir_file + 'sta_in.mat'
savemat(filename, {'lon_sta': lon_stations_in_new, 'lat_sta': lat_stations_in_new})

: 

In [ ]:
R_index = np.arange(5,19,2)
C_index = np.arange(12,54,5)
indexes = []
for row in R_index:
    for col in C_index:
        name = 'R'+str(col).zfill(2)+str(row).zfill(2)
        indexes.append(stalist_all.index(name))
stalist_in = np.array(stalist_all)[indexes]
lon_stations_virsrc = np.zeros_like(np.array(stalist_all)[indexes], dtype=float)
lat_stations_virsrc = np.zeros_like(np.array(stalist_all)[indexes], dtype=float)
for i in range(len(np.array(stalist_all)[indexes])):
    lon, lat = cartesian_to_geographic_2d(x_stations_all[indexes[i]], y_stations_all[indexes[i]])
    lon_stations_virsrc[i] = lon
    lat_stations_virsrc[i] = lat
filename = dir_file + 'sta_virsrc.mat'
savemat(filename, {'lon_sta': lon_stations_virsrc, 'lat_sta': lat_stations_virsrc})

: 

In [ ]:
plt.scatter(lon_stations_in_new, lat_stations_in_new, c='k', label='Original',s = 1)
plt.scatter(lon_stations_virsrc, lat_stations_virsrc, c='r', label='Original',s = 1)

: 

In [ ]:
plt.scatter(x_grid,y_grid,c=vs_inter[:,:,33],cmap='jet')

: 

: 